In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import gc

In [15]:
%reset -f

In [18]:
gc.collect()

480

In [3]:
# Read data file
input_file = '../datasets/mawi_packet_trace_original.csv'
packet_trace = pd.read_csv(input_file)

In [24]:
packet_trace['timestamp'].value_counts()

timestamp
1688187656    131519
1688187664    124474
1688187665    122160
1688187680    120930
1688187604    118928
               ...  
1688187651     89117
1688187623     89055
1688187650     86556
1688187600     85774
1688187692     79855
Name: count, Length: 93, dtype: int64

In [4]:
packet_trace.describe()

,timestamp,src_port,dst_port,protocol,pkt_size
count,9.441776e+06,9.441776e+06,9.441776e+06,9.441776e+06,9.441776e+06
mean,1.688188e+09,1.793662e+04,2.512101e+04,8.800011e+00,5.585487e+02
std,2.701577e+01,2.300241e+04,2.475920e+04,8.236224e+00,9.559149e+02
min,1.688188e+09,0.000000e+00,0.000000e+00,1.000000e+00,5.400000e+01
25%,1.688188e+09,2.660000e+02,4.430000e+02,6.000000e+00,5.800000e+01
50%,1.688188e+09,3.478000e+03,1.320600e+04,6.000000e+00,6.600000e+01
75%,1.688188e+09,4.163100e+04,5.252900e+04,6.000000e+00,1.051000e+03
max,1.688188e+09,6.553500e+04,6.553500e+04,1.320000e+02,2.049400e+04


In [5]:
packet_trace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9441776 entries, 0 to 9441775
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   src_ip     object
 2   dst_ip     object
 3   src_port   int64 
 4   dst_port   int64 
 5   protocol   int64 
 6   pkt_size   int64 
dtypes: int64(5), object(2)
memory usage: 504.2+ MB


In [20]:
def get_duration(start_time, end_time):
    timestamp1 = datetime.fromtimestamp(start_time)
    timestamp2 = datetime.fromtimestamp(end_time)

    # Calculate the duration
    duration = timestamp2 - timestamp1

    if duration.seconds >= 60:
        duration = f'{int(duration.seconds / 60)} mins'
    else:
        duration = f'{int(duration.seconds)} seconds'
    
    return duration

In [8]:
period = get_duration(packet_trace['timestamp'].iloc[0], packet_trace['timestamp'].iloc[-1])

unique_src_ips = len(packet_trace['src_ip'].unique())
unique_dst_ips = len(packet_trace['dst_ip'].unique())

unique_src_ports = len(packet_trace['src_port'].unique())
unique_dst_ports = len(packet_trace['dst_port'].unique())

unique_protocols = len(packet_trace['protocol'].unique())

print(f'Total Time Period: {period}')
print(f'Unique Source IPs: {unique_src_ips}, Unique Destination IPs: {unique_dst_ips}')
print(f'Unique Source Ports: {unique_src_ports}, Unique Destination Ports: {unique_dst_ports}')
print(f'Unique Protocols: {unique_protocols}')

Total Time Period: 1 mins
Unique Source IPs: 122332, Unique Destination IPs: 351872
Unique Source Ports: 65479, Unique Destination Ports: 64919
Unique Protocols: 9


In [93]:
packet_trace.isnull().sum()

timestamp    0
src_ip       0
dst_ip       0
src_port     0
dst_port     0
protocol     0
pkt_size     0
dtype: int64

In [4]:
def get_flows(data, pkt_size_col = 1, elephant_flow_threshold = 100000):
    # Create a dictionary to store flow information
    flows = {}

    pkt_size_columns = {}
    for i in range(2, pkt_size_col+1):
        pkt_size_columns[f'pkt_size_{i}'] = 0
    
    # Iterate through the dataset
    for index, row in data.iterrows():
        # Extract relevant packet attributes 
        timestamp = row['timestamp']
        src_ip = row['src_ip']
        dst_ip = row['dst_ip']
        src_port = row['src_port']
        dst_port = row['dst_port']
        protocol = row['protocol']
        pkt_size = row['pkt_size']

        if protocol == 6 or protocol == 17:
            # Create a unique key for the flow based on the packet attributes
            flow_key = (src_ip, dst_ip, src_port, dst_port, protocol)
    
            # Check if the flow already exists in the dictionary
            if flow_key in flows:
                pkt_count = flows[flow_key]['flow_pkt_count'] + 1
    
                # multi pkt features
                if pkt_size_col > 1 and pkt_count <= pkt_size_col:
                    # update packet size columns
                    flows[flow_key][f'pkt_size_{pkt_count}'] = pkt_size
    
                    # Total size
                    flows[flow_key][f'{pkt_size_col}_pkt_size'] += pkt_size

                    # maximum inter arrival time
                    if flows[flow_key][f'{pkt_size_col}_pkt_max_iat'] < row['timestamp'] - flows[flow_key]['end_time']:
                        flows[flow_key][f'{pkt_size_col}_pkt_max_iat'] = row['timestamp'] - flows[flow_key]['end_time']
    
                    # mean inter arrival time
                    if flows[flow_key]['flow_mean_iat'] > 0:
                        flows[flow_key][f'{pkt_size_col}_pkt_mean_iat'] = np.mean([flows[flow_key]['flow_mean_iat'], (row['timestamp'] - flows[flow_key]['end_time'])])
                    else:
                        flows[flow_key][f'{pkt_size_col}_pkt_mean_iat'] = row['timestamp'] - flows[flow_key]['end_time']
                        
                    # duration
                    flows[flow_key][f'{pkt_size_col}_pkt_duration'] = row['timestamp'] - flows[flow_key]['start_time']
                    
                # flow features
                flows[flow_key]['flow_size'] += pkt_size
                flows[flow_key]['flow_pkt_count'] += 1
    
                # maximum inter arrival time
                if flows[flow_key]['flow_max_iat'] < row['timestamp'] - flows[flow_key]['end_time']:
                    flows[flow_key]['flow_max_iat'] = row['timestamp'] - flows[flow_key]['end_time']
    
                # mean inter arrival time
                if flows[flow_key]['flow_mean_iat'] > 0:
                    flows[flow_key]['flow_mean_iat'] = np.mean([flows[flow_key]['flow_mean_iat'], (row['timestamp'] - flows[flow_key]['end_time'])])                
                else:
                    flows[flow_key]['flow_mean_iat'] = row['timestamp'] - flows[flow_key]['end_time']
                    
                # flow duration
                flows[flow_key]['flow_duration'] = row['timestamp'] - flows[flow_key]['start_time']
    
                # update elephant or mice
                if flows[flow_key]['flow_size'] > elephant_flow_threshold:
                    flows[flow_key]['elephant'] = 1
    
                flows[flow_key]['end_time'] = row['timestamp']
                
            else:
                # Create a new entry for the flow
                if pkt_size_col > 1:
                    flows[flow_key] = {
                        'start_time': timestamp,
                        'end_time': timestamp,
                        'src_ip': src_ip,
                        'dst_ip': dst_ip,
                        'protocol': protocol,
                        'src_port': src_port,
                        'dst_port': dst_port,
                        'pkt_size_1': pkt_size,
                        **pkt_size_columns,
                        f'{pkt_size_col}_pkt_size': pkt_size,
                        f'{pkt_size_col}_pkt_max_iat': 0,
                        f'{pkt_size_col}_pkt_mean_iat': 0,
                        f'{pkt_size_col}_pkt_duration': 0,
                        'flow_size': pkt_size,
                        'flow_pkt_count': 1,
                        'flow_max_iat': 0,
                        'flow_mean_iat': 0,
                        'flow_duration': 0,
                        'elephant': 0,
                    }
    
                else:
                    flows[flow_key] = {
                        'start_time': timestamp,
                        'end_time': timestamp,
                        'src_ip': src_ip,
                        'dst_ip': dst_ip,
                        'protocol': protocol,
                        'src_port': src_port,
                        'dst_port': dst_port,
                        'pkt_size_1': pkt_size,
                        'flow_size': pkt_size,
                        'flow_pkt_count': 1,
                        'flow_max_iat': 0,
                        'flow_mean_iat': 0,
                        'flow_duration': 0,
                        'elephant': 0,
                    }

        
        if index % 100000 == 0:
            print(f'{index} rows processed')
        
    # Output csv file
    # Convert dictionary to dataframe
    data_features = pd.DataFrame.from_dict(flows)

    # Transpose the dataframe
    data_features = data_features.transpose()
    
    # Remove index
#     data_features = data_features.reset_index(drop=True, inplace=True)

    return data_features

In [6]:
flows = get_flows(packet_trace, 7)

# Save dataframe as a csv file without index
output_file = 'flows_4.csv'
flows.to_csv(output_file, index=False)
print('Done')

0 rows processed
100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed
1300000 rows processed
1400000 rows processed
1500000 rows processed
1600000 rows processed
1700000 rows processed
1800000 rows processed
1900000 rows processed
2000000 rows processed
2100000 rows processed
2200000 rows processed
2300000 rows processed
2400000 rows processed
2500000 rows processed
2600000 rows processed
2700000 rows processed
2800000 rows processed
2900000 rows processed
3000000 rows processed
3100000 rows processed
3200000 rows processed
3300000 rows processed
3400000 rows processed
3500000 rows processed
3600000 rows processed
3700000 rows processed
3800000 rows processed
3900000 rows processed
4000000 rows processed
4100000 rows processed
4200000 rows processed
4300000 rows processed
440

In [63]:
# Read data file
input_file = '../datasets/mawi_flows.csv'
flows = pd.read_csv(input_file)

In [64]:
flows

,start_time,end_time,src_ip,dst_ip,protocol,src_port,dst_port,pkt_size_1,pkt_size_2,pkt_size_3,...,7_pkt_size,7_pkt_max_iat,7_pkt_mean_iat,7_pkt_duration,flow_size,flow_pkt_count,flow_max_iat,flow_mean_iat,flow_duration,elephant
0,1688187600,1688187685,149.40.55.233,203.115.138.41,6,1999,54524,66,66,66,...,462,19,12.4375,44,792,12,19,6.982422e+00,85,0
1,1688187600,1688187692,202.11.248.134,52.113.75.222,17,50005,3479,183,191,196,...,1265,0,0.0000,0,841789,4697,1,4.547474e-13,92,1
2,1688187600,1688187606,45.100.248.16,202.249.92.162,6,44734,443,54,54,54,...,216,4,2.5000,6,216,4,4,2.500000e+00,6,0
3,1688187600,1688187600,38.88.114.91,163.37.23.124,6,43332,3128,54,0,0,...,54,0,0.0000,0,54,1,0,0.000000e+00,0,0
4,1688187600,1688187600,163.37.117.124,38.186.128.63,6,61554,443,85,54,0,...,139,0,0.0000,0,139,2,0,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518385,1688187692,1688187692,45.197.187.47,163.37.128.119,6,48152,3128,58,0,0,...,58,0,0.0000,0,58,1,0,0.000000e+00,0,0
1518386,1688187692,1688187692,167.248.189.103,163.37.93.253,6,7528,80,58,0,0,...,58,0,0.0000,0,58,1,0,0.000000e+00,0,0
1518387,1688187692,1688187692,202.249.93.185,46.25.179.131,6,80,12256,66,66,0,...,132,0,0.0000,0,132,2,0,0.000000e+00,0,0
1518388,1688187692,1688187692,146.64.20.95,150.161.104.40,6,44843,2375,54,0,0,...,54,0,0.0000,0,54,1,0,0.000000e+00,0,0


In [54]:
def min_size(s1, s2, s3, s4, s5, s6, s7):
    pkts = [s1, s2, s3, s4, s5, s6, s7]
    pkts = [pkt for pkt in pkts if pkt > 0]

    s_min = np.min(pkts)
    return s_min

In [55]:
def max_size(s1, s2, s3, s4, s5, s6, s7):
    s_max = np.max([s1, s2, s3, s4, s5, s6, s7])
    return s_max

In [56]:
def mean_size(s1, s2, s3, s4, s5, s6, s7):
    pkts = [s1, s2, s3, s4, s5, s6, s7]
    pkts = [pkt for pkt in pkts if pkt > 0]
    
    s_mean = np.mean(pkts)
    return s_mean

In [57]:
flows.insert(14, '7_pkt_min_size', np.vectorize(min_size)(flows['pkt_size_1'],
                                                        flows['pkt_size_2'],
                                                        flows['pkt_size_3'],
                                                        flows['pkt_size_4'],
                                                        flows['pkt_size_5'],
                                                        flows['pkt_size_6'],
                                                        flows['pkt_size_7']))

In [58]:
flows.insert(15, '7_pkt_max_size', np.vectorize(max_size)(flows['pkt_size_1'],
                                                        flows['pkt_size_2'],
                                                        flows['pkt_size_3'],
                                                        flows['pkt_size_4'],
                                                        flows['pkt_size_5'],
                                                        flows['pkt_size_6'],
                                                        flows['pkt_size_7']))

In [59]:
flows.insert(16, '7_pkt_mean_size', np.vectorize(mean_size)(flows['pkt_size_1'],
                                                        flows['pkt_size_2'],
                                                        flows['pkt_size_3'],
                                                        flows['pkt_size_4'],
                                                        flows['pkt_size_5'],
                                                        flows['pkt_size_6'],
                                                        flows['pkt_size_7']))

In [65]:
flows.iloc[:,0:15]

,start_time,end_time,src_ip,dst_ip,protocol,src_port,dst_port,pkt_size_1,pkt_size_2,pkt_size_3,pkt_size_4,pkt_size_5,pkt_size_6,pkt_size_7,pkt_mean_size
0,1688187600,1688187685,149.40.55.233,203.115.138.41,6,1999,54524,66,66,66,66,66,66,66,66.000000
1,1688187600,1688187692,202.11.248.134,52.113.75.222,17,50005,3479,183,191,196,179,171,175,170,180.714286
2,1688187600,1688187606,45.100.248.16,202.249.92.162,6,44734,443,54,54,54,54,0,0,0,54.000000
3,1688187600,1688187600,38.88.114.91,163.37.23.124,6,43332,3128,54,0,0,0,0,0,0,54.000000
4,1688187600,1688187600,163.37.117.124,38.186.128.63,6,61554,443,85,54,0,0,0,0,0,69.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518385,1688187692,1688187692,45.197.187.47,163.37.128.119,6,48152,3128,58,0,0,0,0,0,0,58.000000
1518386,1688187692,1688187692,167.248.189.103,163.37.93.253,6,7528,80,58,0,0,0,0,0,0,58.000000
1518387,1688187692,1688187692,202.249.93.185,46.25.179.131,6,80,12256,66,66,0,0,0,0,0,66.000000
1518388,1688187692,1688187692,146.64.20.95,150.161.104.40,6,44843,2375,54,0,0,0,0,0,0,54.000000


In [61]:
flows

,start_time,end_time,src_ip,dst_ip,protocol,src_port,dst_port,pkt_size_1,pkt_size_2,pkt_size_3,...,7_pkt_size,7_pkt_max_iat,7_pkt_mean_iat,7_pkt_duration,flow_size,flow_pkt_count,flow_max_iat,flow_mean_iat,flow_duration,elephant
0,1688187600,1688187685,149.40.55.233,203.115.138.41,6,1999,54524,66,66,66,...,462,19,12.4375,44,792,12,19,6.982422e+00,85,0
1,1688187600,1688187692,202.11.248.134,52.113.75.222,17,50005,3479,183,191,196,...,1265,0,0.0000,0,841789,4697,1,4.547474e-13,92,1
2,1688187600,1688187606,45.100.248.16,202.249.92.162,6,44734,443,54,54,54,...,216,4,2.5000,6,216,4,4,2.500000e+00,6,0
3,1688187600,1688187600,38.88.114.91,163.37.23.124,6,43332,3128,54,0,0,...,54,0,0.0000,0,54,1,0,0.000000e+00,0,0
4,1688187600,1688187600,163.37.117.124,38.186.128.63,6,61554,443,85,54,0,...,139,0,0.0000,0,139,2,0,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518385,1688187692,1688187692,45.197.187.47,163.37.128.119,6,48152,3128,58,0,0,...,58,0,0.0000,0,58,1,0,0.000000e+00,0,0
1518386,1688187692,1688187692,167.248.189.103,163.37.93.253,6,7528,80,58,0,0,...,58,0,0.0000,0,58,1,0,0.000000e+00,0,0
1518387,1688187692,1688187692,202.249.93.185,46.25.179.131,6,80,12256,66,66,0,...,132,0,0.0000,0,132,2,0,0.000000e+00,0,0
1518388,1688187692,1688187692,146.64.20.95,150.161.104.40,6,44843,2375,54,0,0,...,54,0,0.0000,0,54,1,0,0.000000e+00,0,0


In [66]:
flows.rename(columns = {'pkt_mean_size':'7_pkt_mean_size'}, inplace = True)
flows.rename(columns = {'pkt_max_size':'7_pkt_max_size'}, inplace = True)
flows.rename(columns = {'pkt_min_size':'7_pkt_min_size'}, inplace = True)

In [68]:
flows.iloc[:,0:15]

,start_time,end_time,src_ip,dst_ip,protocol,src_port,dst_port,pkt_size_1,pkt_size_2,pkt_size_3,pkt_size_4,pkt_size_5,pkt_size_6,pkt_size_7,7_pkt_mean_size
0,1688187600,1688187685,149.40.55.233,203.115.138.41,6,1999,54524,66,66,66,66,66,66,66,66.000000
1,1688187600,1688187692,202.11.248.134,52.113.75.222,17,50005,3479,183,191,196,179,171,175,170,180.714286
2,1688187600,1688187606,45.100.248.16,202.249.92.162,6,44734,443,54,54,54,54,0,0,0,54.000000
3,1688187600,1688187600,38.88.114.91,163.37.23.124,6,43332,3128,54,0,0,0,0,0,0,54.000000
4,1688187600,1688187600,163.37.117.124,38.186.128.63,6,61554,443,85,54,0,0,0,0,0,69.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518385,1688187692,1688187692,45.197.187.47,163.37.128.119,6,48152,3128,58,0,0,0,0,0,0,58.000000
1518386,1688187692,1688187692,167.248.189.103,163.37.93.253,6,7528,80,58,0,0,0,0,0,0,58.000000
1518387,1688187692,1688187692,202.249.93.185,46.25.179.131,6,80,12256,66,66,0,0,0,0,0,66.000000
1518388,1688187692,1688187692,146.64.20.95,150.161.104.40,6,44843,2375,54,0,0,0,0,0,0,54.000000


In [69]:
# Save dataframe as a csv file without index
output_file = '../datasets/mawi_flows.csv'
flows.to_csv(output_file, index=False)
print('Done')

Done
